In [2]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
from IPython.core.pylabtools import figsize
from IPython.display import IFrame
from pandas.api.types import CategoricalDtype
from pandas import merge
from scipy.sparse import csr_matrix
from sklearn import datasets
from sklearn import metrics
from sklearn import neighbors                           
from sklearn import tree, linear_model,neighbors   
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.datasets import load_boston
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import*
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [6]:
# Retrive our sales table.
sales = pd.read_csv('Daily Wholegood Sales Master.csv')

In [28]:
#jdquote 1 is original jdquote attached with the column of Base/Implement and combined CustomerName
jdquote1 = pd.read_csv('jdquote1.csv')

/Applications/anaconda3/envs/cis432/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,15,36,49,52,56,57,58,59,61,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55377 entries, 0 to 55376
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Location          55377 non-null  object 
 1   Stock #           55377 non-null  int64  
 2   Serial No         52114 non-null  object 
 3   Model             55377 non-null  object 
 4   Make              55377 non-null  object 
 5   Description       55320 non-null  object 
 6   New/Used          55377 non-null  object 
 7   Sale Date         55377 non-null  object 
 8   Date Rcvd         55322 non-null  object 
 9   Sold To           53546 non-null  object 
 10  Total Sales       55377 non-null  object 
 11  Total Cost        55377 non-null  object 
 12  Gross Margin      55377 non-null  object 
 13  Gross Margin %    53788 non-null  object 
 14  salesperson_name  55374 non-null  object 
 15  Salesperson Code  55374 non-null  object 
 16  Acquired By       10294 non-null  object

In [9]:
jdquote1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330451 entries, 0 to 330450
Data columns (total 67 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   330451 non-null  int64  
 1   QuoteID                      330431 non-null  object 
 2   NameOfQuote                  274895 non-null  object 
 3   QuoteType                    330431 non-null  object 
 4   CreationDate                 330431 non-null  object 
 5   ExpirationDate               330069 non-null  object 
 6   CurrentQuoteStatus           330431 non-null  object 
 7   CreatedLogon                 330431 non-null  object 
 8   LastModifiedLogon            330431 non-null  object 
 9   SalesPersonName              330415 non-null  object 
 10  CompanyName                  39667 non-null   object 
 11  ContactLastName              192953 non-null  object 
 12  ContactFirstName             191408 non-null  object 
 13 

In [10]:
# We only want records of non-null stocknumbers.
jdquote1 = jdquote1.dropna(subset = ['StockNumber'])
jdquote1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74212 entries, 0 to 330450
Data columns (total 67 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   74212 non-null  int64  
 1   QuoteID                      74212 non-null  object 
 2   NameOfQuote                  47391 non-null  object 
 3   QuoteType                    74212 non-null  object 
 4   CreationDate                 74212 non-null  object 
 5   ExpirationDate               74144 non-null  object 
 6   CurrentQuoteStatus           74212 non-null  object 
 7   CreatedLogon                 74212 non-null  object 
 8   LastModifiedLogon            74212 non-null  object 
 9   SalesPersonName              74211 non-null  object 
 10  CompanyName                  14382 non-null  object 
 11  ContactLastName              52516 non-null  object 
 12  ContactFirstName             52473 non-null  object 
 13  City           

In [11]:
# Retrive our Stock-joint-table.csv.
#"Stock-joint-table.csv" is inner join table on stocknumber of jdquote1 and sales table
# with deletion of unmatched records of both customername columns by filtering out fuzzywuzzy score less than 80%
# and replace two columns of customername with same name
stock_joint_table = pd.read_csv('Stock-joint-table.csv')

/Applications/anaconda3/envs/cis432/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,15,49,52,56,59,61,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# We only want records of non-null stocknumbers.
stock_joint_table1 = stock_joint_table.drop_duplicates('StockNumber', keep='last')

In [74]:
stock_joint_table1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40258 entries, 1 to 69548
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   X                            40258 non-null  int64  
 1   QuoteID                      40258 non-null  object 
 2   NameOfQuote                  24152 non-null  object 
 3   QuoteType                    40258 non-null  object 
 4   CreationDate                 40258 non-null  object 
 5   ExpirationDate               40246 non-null  object 
 6   CurrentQuoteStatus           40258 non-null  object 
 7   CreatedLogon                 40258 non-null  object 
 8   LastModifiedLogon            40258 non-null  object 
 9   SalesPersonName              40257 non-null  object 
 10  CompanyName                  7432 non-null   object 
 11  ContactLastName              32345 non-null  object 
 12  ContactFirstName             32336 non-null  object 
 13  City            

In [13]:
sales_ljoint_innerstk = pd.merge(sales,stock_joint_table1,left_on='Stock #',right_on='StockNumber',how = 'left')

In [14]:
# I choose this "sales_ljoint_innerstk" as my final sales table to do descriptive analysis,
# so that I could have more detailed information and correct matched customer name.
sales_ljoint_innerstk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55377 entries, 0 to 55376
Columns: 106 entries, Location_x to sold_to_code_y
dtypes: float64(19), int64(1), object(86)
memory usage: 45.2+ MB


In [325]:
sales_ljoint_innerstk.to_csv('sales_ljoint_innerstk.csv')

In [15]:
sales_ljoint_nonamargin = sales_ljoint_innerstk.dropna(subset=["Gross Margin %"])

In [ ]:
sales_ljoint_innerstk_nonamargin.to_csv(r'/Users/wanbing/Documents/SIMON/SIMON/CIS465/Project landpro/draw/sales_ljoint_innerstk_nonamargin.csv', index = False, header = True)

In [29]:
#Construct a table shows the frequency of Equipment Category in both "jdquote1" and "sales_ljoint_innerstk" level
# and ratio of frequencies as generation rate

#1. create frequency column of "sales_ljoint_innerstk"
salescatfreq = sales_ljoint_innerstk['EquipmentCategoryDesc'].value_counts()
salescatfreq = salescatfreq.to_frame()
salescatfreq = salescatfreq.reset_index()
salescatfreq = salescatfreq.rename({'index':'Category', 'EquipmentCategoryDesc':'freq'}, axis='columns')


#2. create frequency column of "jdquote1"
jdcatfreq = jdquote1['EquipmentCategoryDesc'].value_counts()
jdcatfreq = jdcatfreq.to_frame()
jdcatfreq = jdcatfreq.reset_index()
jdcatfreq = jdcatfreq.rename({'index':'Category', 'EquipmentCategoryDesc':'freq'}, axis='columns')

#salescatfreq
#jdcatfreq

#3. create ratio column of 2 frequencies
salescatfreq['ratio'] = salescatfreq['freq']/jdcatfreq['freq']
salescatfreq['freqx'] = salescatfreq['freq']
salescatfreq['freqy'] = jdcatfreq['freq']
salescatfreq

,Category,freq,ratio,freqx,freqy
0,Mowers,15595,0.123179,15595,126604
1,AG Tractors,6847,0.170502,6847,40158
2,Turf implements,2835,0.087284,2835,32480
3,Material Handling,2176,0.083957,2176,25918
4,Utility Vehicles,2107,0.093247,2107,22596
5,Snow Equipment,1657,0.085753,1657,19323
6,Landscape Equipment,1404,0.111730,1404,12566
7,Agriculture Implements,1126,0.121980,1126,9231
8,Cutters & Shredders,1025,0.169674,1025,6041
9,Hay & Forage,824,0.177586,824,4640


In [30]:
#Construct a table shows the frequency of states in both "jdquote1" and "sales_ljoint_innerstk" level
# and ratio of frequencies as generation rate

#1. create frequency column of "sales_ljoint_innerstk"
salesstatefreq = sales_ljoint_innerstk['StateProvince'].value_counts()
salesstatefreq = salesstatefreq.to_frame()
salesstatefreq = salesstatefreq.reset_index()
salesstatefreq = salesstatefreq.rename({'index':'StateProvince', 'StateProvince':'freq'}, axis='columns')
 

#2. create frequency column of "jdquote1"
jdstatefreq = jdquote1['StateProvince'].value_counts()
jdstatefreq = jdstatefreq.to_frame()
jdstatefreq = jdstatefreq.reset_index()
jdstatefreq = jdstatefreq.rename({'index':'StateProvince', 'StateProvince':'freq'}, axis='columns')

#salesstatefreq 
#jdstatefreq

#3. create ratio column of 2 frequencies
salesstatefreq['ratio'] = salesstatefreq['freq']/jdstatefreq['freq']
salesstatefreq['freqx'] = salesstatefreq['freq']
salesstatefreq['freqy'] = jdstatefreq['freq']
salesstatefreq


,StateProvince,freq,ratio,freqx,freqy
0,NY,24555,0.202559,24555,121224
1,PA,13412,0.130014,13412,103158
2,OH,839,0.157146,839,5339
3,ON,72,0.331797,72,217
4,MI,50,0.324675,50,154
5,SD,42,0.287671,42,146
6,FL,32,0.275862,32,116
7,WV,30,0.267857,30,112
8,SC,26,0.260000,26,100
9,NJ,24,0.250000,24,96


In [49]:
#Save 2 frequency generation tables: one for equipment type level, one for state level.
salesstatefreq.to_csv(r'/Users/wanbing/Documents/SIMON/SIMON/CIS465/Project landpro/draw/stateratio.csv', index = False, header = True)
salescatfreq.to_csv(r'/Users/wanbing/Documents/SIMON/SIMON/CIS465/Project landpro/draw/catratio.csv',index=False,header=True)